# Experimental Results 1: SolarBatteryHouse

To start of we run some experiments on the default version of the main Bauwerk environment, `SolarBatteryHouse`. We recommend rerunning these experiments as baselines for work using this environment.

## Evaluation

There is no one way that performance on the `SolarBatteryHouse` can or should be measured. One obvious choice would be to directly compare the cumulative reward, i.e. overall grid energy payments. However, the magnitude of this value completely depends on the configuration of the environment. Thus, this can make it difficult to compare performance between environments, even within Bauwerk. Instead, we propose using the access to the optimal control actions by considering *performance relative to random and optimal control*,

$p' = \frac{p_m-p_r}{p_o-p_r}$,

where $p_m$ is the average reward of the method to be evaluated, $p_r$ and $p_o$ the average rewards of random and optimal control respectively. A value $>0$ means that the method is better than random, a value close to $1$ means that the method is close to optimal, a value $<0$ means that the method is worse than random, i.e. completely useless.



## Baselines

### Random actions

With this let's get started by establishing the lowest bar: what's the performance we get when we just take random actions in the environment (sampled from the action space). To do this we first need to determine how many actions in the environment we want to evaluate over (`EVAL_LEN`) and setup some helper code.

In [3]:
# Setup and helper code
import bauwerk
import gym
import numpy as np

EVAL_LEN = 24*30 # evaluate on 1 month of actions

# Create SolarBatteryHouse environment
env = gym.make("bauwerk/SolarBatteryHouse-v0")

def evaluate_actions(actions, env):
    cum_reward = 0
    obs = env.reset()
    for action in actions:
        obs, reward, done, info = env.step(np.array(action, dtype=np.float32))
        cum_reward += reward
    
    return cum_reward / len(actions)

In [9]:
# mean random performance over 100 trials
random_trials = [evaluate_actions([env.action_space.sample() for _ in range(EVAL_LEN)], env) for _ in range(100)]
random_std = np.std(random_trials)
p_rand = np.mean(random_trials)
# note: std here is between different trials (of multiple actions)
print(f"Avg reward with random actions: {p_rand} (standard deviation: {random_std})")

Avg reward with random actions: -0.5455792138136959 (standard deviation: 0.025515726632037904)


### Optimal actions

`SolarBatteryHouse` is a fully tractable environment, thus Bauwerk can easily compute the theoretically optimal actions one can take in the environment.

In [22]:
optimal_actions, _ = bauwerk.solve(env)
p_opt = evaluate_actions(optimal_actions.reshape((-1,1))[:EVAL_LEN], env)
print(f"Avg reward (per step) with optimal actions: {p_opt}")

Avg reward (per step) with optimal actions: -0.10358608226144295


## Reinforcement learning agent

Next we consider a simple reinforcement learning (RL) agent. We use [Stable Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) to access RL algorithm implementations.

Coming soon.